In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [32]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import sentence_transformers
from langchain.chains import ConversationalRetrievalChain
from tqdm import tqdm
from langchain.llms import Together
from langchain.prompts import PromptTemplate

In [2]:
embedding = HuggingFaceEmbeddings(model="BAAI/bge-large-en-v1.5" ,encode_kwargs={"normalize_embeddings": True})
db = FAISS.load_local("faiss_index", embedding,allow_dangerous_deserialization=True)

In [33]:
loader = PyMuPDFLoader("HR-policy-text.pdf")
documents = loader.load()
print(documents[15].page_content[:500]) 

431278/2022/SPV DIVISION
16
Code of Conduct
1.1 
Title and Commencement
This Policy will be called the ‘NHLML Code of Conduct’. The Policy comes into
force effective from
.
1.2 
Applicability
Except as otherwise provided by or under these rules they shall apply to all persons
appointed against regular posts in connection with the affairs of the NHLML. These rules
shall not apply to appointments on casual basis.
1.3 
Objectives
The purpose of this is to define a Service Code of Conduct for all Ex


In [34]:
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=100)
docs = splitter.split_documents(documents)

In [35]:
embeddings = HuggingFaceEmbeddings(model="BAAI/bge-large-en-v1.5" ,encode_kwargs={"normalize_embeddings": True}) #thenlper/gte-small 
db = FAISS.from_documents(docs, embeddings)


_ = [embeddings.embed_query(doc.page_content) for doc in tqdm(docs)]

100%|██████████| 75/75 [02:08<00:00,  1.71s/it]


In [53]:
db.save_local("faiss_index")

In [36]:
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
import os

load_dotenv()
api_key = os.getenv("together_api_key")

In [ ]:

llm = Together(
    model="lgai/exaone-3-5-32b-instruct",
    temperature=0,
    together_api_key=api_key
)

In [61]:
retriever = db.as_retriever(search_kwargs={"k": 3, "fetch_k": 10})
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an intelligent and professional HR assistant for a corporate organization named National Highways Logistics Management Ltd (NHLML) and you are named is Tara.
You are talking to the employee of NHLML about whom you don't know anything except this.

Your responsibilities include:
- Answering the question according to the policy
- Do not answer questions which are not related to HR or company policies.(e.g., food, pets, travel, entertainment,or any such casual request etc.).
- In case of unsurety,just reply with "Sorry that is not mentioned in the policy".
- Do not respond after simple acknowledgments like "okay", "thanks", "ok", etc.


Context:
{context}

Question:
{question}

Answer:
"""
)

In [62]:

ConversationBufferWindowMemory(k=2)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    k=2
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  
    retriever=retriever,
    combine_docs_chain_kwargs={"prompt": prompt},
    memory=memory
)


In [63]:
qa_chain.memory.clear()

In [ ]:
query = "what is maternity leave"
response = qa_chain.run(query)
print(prompt)

# print("Bot:", response)

ValueError: Together received an invalid payload: {
  "id": "o2VSRnn-2kFHot-95cedf298c7e8adc-PDX",
  "error": {
    "message": "You have reached the rate limit specific to this model meta-llama/Llama-3.3-70B-Instruct-Turbo-Free. The maximum rate limit for this model is 6.0 queries and 180000000 tokens per minute. This limit differs from the general rate limits published at Together AI rate limits documentation (https://docs.together.ai/docs/rate-limits). For inquiries about increasing your model-specific rate limit, please contact our sales team (https://www.together.ai/forms/contact-sales)",
    "type": "model_rate_limit",
    "param": null,
    "code": null
  }
}

In [16]:
query = "generate me meta data for product KOORSI & COMPANY. Aaram Mid Back Mesh Ergonomic Office Chair, Tilting and Height Adjustable Mechanism, 6 Months Limited Warranty, Revolving Chair for Home & Office (Black)"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
**Product Metadata for KOORSI & COMPANY - Aaram Mid Back Mesh Ergonomic Office Chair**

**Product Name:**
Aaram Mid Back Mesh Ergonomic Office Chair, Tilting and Height Adjustable Mechanism

**Brand:**
KOORSI & COMPANY

**Model:**
Aaram

**Type:**
Ergonomic Office Chair

**Key Features:**
- **Mesh Back Design:** Promotes breathability and comfort
- **Ergonomic Design:** Supports proper posture and reduces strain
- **Tilting Mechanism:** Adjustable recline for personalized comfort
- **Height Adjustable:** Customizable seat height to fit various users
- **Revolving Base:** 360-degree swivel for easy mobility
- **Suitable for:** Home and Office use

**Color Options:**
- Black

**Warranty:**
- **Limited Warranty:** 6 Months

**Dimensions:**
- (Specify dimensions if available, e.g., Seat Height: [X] cm - [Y] inches, Seat Width: [X] cm - [Y] inches, Back Height: [X] cm - [Y] inches)

**Weight Capacity:**
- (Specify weight capacity if available, e.g., Up to [X] kg/[Y] lbs)

**Materials:

In [8]:
query = "how many sick leaves can i get"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
The provided document does not specify the number of sick leaves (Casual Leave) an executive is entitled to. It mainly focuses on other types of leave such as Extra-ordinary Leave (EOL/SL), Special Disability Leave, and Maternity Leave, without detailing the entitlements for Casual Leave or sick leave specifically. For accurate information regarding the number of sick leaves you are entitled to, you would need to refer to the specific leave policy section that addresses Casual Leave or contact your HR department directly.


In [37]:
query = "what  are the working hours"
response = qa_chain.run(query)
print("Bot:", response)

Bot: The provided context does not contain specific information regarding the standard working hours of the company. Therefore, I cannot provide an accurate answer based solely on the given information.


In [38]:
query = "which one can i opt for"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
If you need time off for personal reasons, you can opt for **Casual Leave**. According to the provided context, casual leave is specifically intended to cover casual absences of executives for personal reasons, with a maximum allowance of 8 days in a calendar year.


In [39]:
query = "how many leaves am i entitled to in a year as an employee"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
Based on the provided context, here is the breakdown of your total leave entitlements in a calendar year:

1. **Casual Leave**: 8 days
2. **Earned Leave (EL)**: 30 days credited in advance, with a mandatory requirement to take at least 10 days EL in a calendar year. You can carry over a maximum of 20 EL days to the next year.
3. **Maternity Leave**: Up to 180 days (for female executives with less than 2 surviving children) or up to 45 days (for any female executive regardless of the number of children). This leave is not debited against the regular leave account.

**Note**: The context does not specify a fixed number of sick leaves (other than what might be covered under Maternity Leave or Special Disability Leave), so we will not include an additional specific number for sick leave unless explicitly stated.

**Total Entitlements (excluding sick leave, as it's not quantified)**:
- Casual Leave: 8 days
- Earned Leave: 30 days (with a minimum usage requirement of 10 days)
- Materni

In [10]:
query = "resignation policy"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
The company's policy regarding resignation is as follows:

1. **Submission of Resignation:** An executive must submit a resignation letter in writing or via email to the Reporting Officer, with a copy sent to HR.

2. **Relief from Duties:** The executive will not be relieved of their duties until their resignation is accepted and officially communicated to all concerned parties.

3. **Effective Date:** The executive's employment will cease on the date specified in the resignation letter.

4. **Notice Period:** A notice period is essential to ensure a smooth handover of responsibilities. The notice period varies based on the category of the executive:
   - **Probationary Executives:** One month or salary in lieu thereof.
   - **Regular Executives:** Two months or salary in lieu thereof.
   - **Temporary/Contract Executives:** As per the terms and conditions of their appointment/contract.

5. **Settlement of Dues:** Upon acceptance of the resignation and agreement on a relieving da

In [41]:
query = "what if i want to do it today itself"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
Based on the provided context, resigning effective immediately is not explicitly detailed, but we can infer the following procedures and considerations:

1. **Formal Resignation Submission**: You must still submit a formal resignation letter in writing or via email to your Reporting Officer, along with a copy to HR, even if immediate resignation is desired.

2. **Notice Period Waiver**: Since immediate resignation is not standard, you would need to request a waiver of the notice period. This request should be justified clearly, explaining the reasons for the immediate resignation.

3. **Approval Process**:
   - **For Senior Management Positions (VP & above)**: The Chairman would need to approve the waiver and immediate resignation.
   - **For Middle and Junior Management Positions (AVP & below)**: The CEO would need to approve the waiver and immediate resignation.

4. **Handover and Dues**: Despite the immediate resignation, you are still expected to:
   - Ensure a smooth handove

In [42]:
query = "what does the chapter on resignation say "
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
The chapter on resignation policies specifically outlines the following key points:

1. **Resignation Submission**: An executive must submit a written (or email) resignation letter to the Reporting Officer, with a copy to HR, providing the requisite notice period.

2. **Acceptance of Resignation**: The resignation is not considered final until it is accepted by the Reporting Officer, and this acceptance must be communicated to all concerned parties.

3. **Notice Period**:
   - **Probation**: One month or salary in lieu thereof.
   - **Regular Executives**: Two months or salary in lieu thereof.
   - **Temporary/Contract Executives**: As per the terms and conditions of their appointment/contract.
   - **Waiver**: The notice period may be waived at the discretion of the Head of Department (HoD) with approval from the Chairman, subject to justification by the executive.

4. **Duties During Notice Period**: The executive must clear all dues to the organization, hand over responsibilit

In [43]:
query = "how to take maternity leave"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
To proceed with taking maternity leave according to the company's policy, follow these steps:

1. **Eligibility Check**: Ensure you are eligible for maternity leave. You must be a female executive with less than two surviving children.

2. **Application Submission**:
   - Submit an application to the sanctioning authority well in advance. For maternity leave, it is recommended to apply at least 7 days before the intended start date of your leave, unless it is an emergency.
   - Provide a medical certificate from an authorized medical attendant confirming your pregnancy and the expected due date.

3. **Leave Duration**:
   - You are entitled to 180 days of maternity leave from the date of commencement, which can be extended up to 24 months in case of complications or as per medical advice.
   - Maternity leave can also be combined with earned leave on medical certificate, but this should not exceed 45 days in total additional leave.

4. **Leave Type**:
   - Maternity leave is gran

In [44]:
query = "can you write an application for maternity leave for me according to the company policies"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
**Application for Maternity Leave**

**To:** [Sanctioning Authority's Name]  
[Sanctioning Authority's Position]  
[Company's Name]  
[Company's Address]

**From:** [Your Full Name]  
[Your Position]  
[Department]  
[Company's Name]  
[Your Contact Information]

**Subject:** Application for Maternity Leave

**Date:** [Current Date]

---

**Dear [Sanctioning Authority's Name],**

I am writing to formally apply for maternity leave in accordance with the company's leave policy as outlined in the HR guidelines.

**Details of Leave Request:**

- **Employee Name:** [Your Full Name]
- **Employee ID:** [Your Employee ID]
- **Position:** [Your Position]
- **Department:** [Your Department]
- **Expected Date of Commencement of Leave:** [Expected Start Date of Maternity Leave]
- **Duration of Leave Requested:** 180 days from the date of commencement, subject to medical certification.
- **Reason for Leave:** Maternity leave following the expected delivery of my child.

**Supporting Documents

In [45]:
query = "how many leaves can i get according to it"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
According to the company's policies, the total number of leaves an executive is entitled to in a calendar year, including maternity leave specifics, can be broken down as follows:

1. **Casual Leave**: 8 days per calendar year.
2. **Earned Leave (EL)**: 30 days per calendar year, with 15 days credited in advance on January 1st and July 1st.
3. **Maternity Leave**: 
   - Up to 180 days for female executives with less than 2 surviving children, subject to medical certification.
   - Additionally, maternity leave can be combined with earned leave, but the specific additional days from earned leave are not detailed beyond the standard 30 days per year.

It's important to note that the combination of maternity leave with earned leave can increase the total leave period, but the base entitlement without combination is:

- **Base Entitlement**: 8 (Casual) + 30 (Earned) = 38 days.
- **Maternity Leave**: Up to 180 days, which is a significant period but is not typically counted within the

In [46]:
query = "what is the recipe to cook rice"
response = qa_chain.run(query)
print("Bot:", response)

Bot: I'm sorry, but I cannot provide recipes or information outside the scope of the corporate policies and documents provided. The context given focuses on medical health check-up policies, leave rules, and domestic travel policies, and does not include any culinary information. For cooking instructions, you may want to refer to a reliable cookbook or a trusted online recipe source.


In [47]:
query = "what are the break timings per day"
response = qa_chain.run(query)
print("Bot:", response)

Bot: The provided document does not specify exact break timings for each workday. However, it does mention that for casual leave taken for half a day, "the lunch interval will be taken as the dividing line." This implies that there is a lunch break, but the start and end times of this break are not detailed in the given context. For precise break timings, further company-specific HR guidelines or policies would need to be consulted.


In [48]:
query = "how to train a dog"
response = qa_chain.run(query)
print("Bot:", response)

Bot: I'm sorry, but I cannot provide information on training dogs as it falls outside the scope of the provided corporate policy document which focuses on employee conduct, property management, and other organizational guidelines within NHLML. For information on dog training, you may want to consult resources specifically dedicated to animal training or behavior.


In [49]:
query = "i want paternity leave"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
According to the company's policies, the entitlements and procedures for Paternity Leave are as follows:

**Entitlements:**
- A male executive with less than two surviving children is eligible for Paternity Leave on full pay.
- The duration of Paternity Leave is 15 days during the confinement of his wife.
- This leave can be availed up to 15 days before or up to six months from the date of delivery of the child.
- Paternity Leave must be supported by documentary evidence.
- If not availed, the leave will lapse.

**Procedures:**
- The executive must apply for Paternity Leave through the proper channel, providing necessary documentation to support the leave request.
- The application should be made sufficiently in advance to ensure timely processing.
- There is no specific mention of a minimum notice period for Paternity Leave in the provided context, but generally, following the broader leave application guidelines (e.g., at least 24 hours for leave of 3 days or less, and 7 days f

In [50]:
query = "how many paternity leaves do i get?"
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
According to the company policy, a male executive with less than two surviving children is entitled to Paternity Leave on full pay for a period of 15 days during the confinement of his wife, based on documentary evidence.


In [51]:
query = "which places can i visit in delhi "
response = qa_chain.run(query)
print("Bot:", response)

Bot: I'm unable to provide information on popular tourist destinations or travel recommendations for Delhi or any other location, as the given context is focused on the internal travel policy and compensation structure of NHLML, without including details about tourist attractions. For travel recommendations, you might want to refer to travel guides or official tourism websites.


In [52]:
query = "tell me about benefits i avail "
response = qa_chain.run(query)
print("Bot:", response)

Bot: 
Based on the provided context, as an employee, you are entitled to the following benefits according to the company policies:

1. **Group Life Insurance Policy**:
   - Coverage in case of death or permanent disability, with a sum assured as specified in the table, payable to your nominated family member(s).

2. **Group Medical Policy**:
   - Comprehensive medical cover including out-patient and specialized treatments in approved hospitals.
   - Coverage for:
     - Pathological, radiological, and other diagnostic methods.
     - Consultations at clinics or with registered medical practitioners.
     - Home treatment if certified necessary by a registered medical practitioner.
     - Outdoor patient treatment in approved hospitals.
     - Supply of medicines, vaccines, serums, and other therapeutic substances.
   - The "family" definition includes your spouse, parents, unmarried or widowed daughters/sons below 25 years, and fully dependent minor siblings.

3. **Leave Policy**:
   -